In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
# %pprint

In [2]:
import glob
import sys,os
import pandas as pd
import os
# import dicom
import SimpleITK as sitk
import ipywidgets as widgets
import matplotlib.mlab as mlab
import matplotlib.patches as patches
import numpy as np
import pandas as pd
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.stats import norm
from tqdm import tqdm
pd.set_option('display.max_colwidth', 500)
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from ipywidgets import interactive, widgets, Layout, Box
from IPython.display import display
from IPython.display import clear_output
from scipy.signal import savgol_filter
import shutil
import ast
import matplotlib
matplotlib.style.use('ggplot')
plt.style.use('seaborn-poster')
import re
for path in ['..','../..']:
    sys.path.append(path)
import settings as S

In [3]:
info_csv= pd.read_csv(os.path.join(S.root,"sheets","info.csv"))
patients=info_csv['patient id'].unique()

In [4]:
excluded_patients=[]

In [5]:
patients=[patient for patient in patients if patient not in excluded_patients]

In [6]:
T2_not_desired=['map','opt','RSSFSE','sub','calibration','body','Opt','fov','reformated','3pl','loc','pjn','screen','Cube','trace','WATER','phase','ideal','trufi','S6_ND']

# T2

In [7]:
ax_list=[]
sag_list=[]
cor_list=[]

for patient in patients:
# patient=patients[0]
    
    series=info_csv.loc[info_csv['patient id']==patient,'series description'].values
#     patient_folder=os.path.join(os.path.dirname(S.raw_folder),'raw_sorted',str(patient).zfill(8))
#     study=os.listdir(patient_folder)
#     series=os.listdir(os.path.join(patient_folder,study[0]))
    T2s=[serie for serie in series if 'T2' in serie and not any(key.upper() in serie.upper() for key in T2_not_desired)]
    
#     if len(T2s)!=3:
#         print(patient)
#         print(T2s)
    AX=[serie for serie in T2s if 'AX' in serie.upper()] 
    SAG=[serie for serie in T2s if 'SAG' in serie.upper()]
    COR=[serie for serie in T2s if 'COR' in serie.upper()]
    
    if len(COR)>1:
        print(patient)
        print(COR)
        
#         print(T2s)
#     if len(AX)>1:
#         AX=[serie for serie in T2s if 'REPEAT' in serie.upper()][0]
#     else:
#         AX=AX[0]
#     ax_list.append(AX)
    
#     if len(SAG)>1:
#         SAG=[serie for serie in T2s if 'REPEAT' in serie.upper()][0]
#     else:
#         SAG=SAG[0]
#     sag_list.append(SAG)
    
#     if len(COR)>1:
#         COR=[serie for serie in T2s if 'REPEAT' in serie.upper()][0]
#     else:
#         COR=COR[0]
#     cor_list.append(COR)

8708521
['COR FRFSE-XL T2', 'COR FRFSE-XL T2']
23602410
['COR FRFSE-XL T2', 'COR FRFSE-XL T2']
23742679
['COR FRFSE-XL T2', 'COR FRFSE-XL T2']
26330787
['COR FRFSE-XL T2', 'COR FRFSE-XL T2']
27862614
['COR FRFSE-XL T2', 'COR FRFSE-XL T2']


# DWI

In [8]:
dwi_info_df= pd.read_csv(os.path.join(S.root,"sheets","dwi_stats.csv"))
dwi_only_info_df=dwi_info_df.loc[~dwi_info_df['series description'].str.contains('ADC|Apparent',case=False)]

In [9]:
dwi_only_info_df['b values']=dwi_only_info_df['b values'].apply(lambda x: ast.literal_eval(x))

/home/mehrtash/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
DWI_not_desired=T2_not_desired+['b3500','DWI 12 B','COR','PU','Focus']

In [ ]:
info_csv.loc[((info_csv['series description'].str.contains('1000')))&(info_csv['series description'].str.contains('DWI'))&(~info_csv['series description'].str.contains('ADC'))&(~info_csv['series description'].str.contains('|'.join(DWI_not_desired),case=False))].groupby('series description')['patient id'].nunique()

In [ ]:
DWIs=set([serie for serie in info_csv["series description"].values if 'DWI' in serie and not 'ADC' in serie and not any(key.upper() in serie.upper() for key in DWI_not_desired)])
DWIs

In [ ]:
print(list(map(int, re.findall(r'\d+', list(DWIs)[6]))))
print(list(DWIs)[6])

In [ ]:
dwi_only_info_df.loc[dwi_only_info_df['b values'].map(len)!=2]

In [ ]:
dwi_only_info_df['b values str']=dwi_only_info_df['b values'].apply(lambda x: str(x))
dwi_only_info_df.groupby('b values str')['patient id'].count()

In [ ]:
dwi_only_info_df.loc[dwi_only_info_df['b values str'].str.contains('1000')]

In [ ]:
dwi_only_info_df.loc[dwi_only_info_df['patient id']==20826814]

# ADC

In [ ]:
dwi_info_all=pd.read_csv(os.path.join(S.root,"sheets","dwi_stats_all.csv"))

In [ ]:
dwi_info_all.loc[dwi_info_all['series description'].str.contains('pu|opt',case=False),'patient id'].unique()

In [ ]:
dwi_info_all.loc[dwi_info_all['actual time stamp'].isnull()]

In [ ]:
adc_info_df=dwi_info_df.loc[(dwi_info_df['series description'].str.contains('ADC|Apparent',case=False))]

In [ ]:
adc_info_df.loc[adc_info_df['b values'].isnull()].groupby('series description')['patient id'].count()

In [ ]:
adc_not_null_df=adc_info_df.loc[(~adc_info_df['b values'].isnull())]
adc_null_df=adc_info_df.loc[adc_info_df['b values'].isnull()]

In [ ]:
adc_not_null_df['b values str']=adc_not_null_df['b values'].apply(lambda x: str(x))
adc_not_null_df.groupby('b values str')['patient id'].count()

In [ ]:
adc_null_patients = adc_null_df['patient id'].unique()

In [ ]:
adc_not_null_df.loc[(~adc_not_null_df['b values'].str.contains('500|1400'))]

In [ ]:
dwi_info_df.loc[dwi_info_df['patient id']==23602410]

# big dic

In [ ]:
big_dic_df= pd.read_csv(os.path.join(S.root,"sheets","dwi_stats_big_dic.csv"))

In [ ]:
str('123').startswith('1')

In [ ]:
dic=big_dic_df['bval_big_dic'][0]

In [ ]:
dic

In [ ]:
[key1 for key1 in dic.keys() if (dic[key1]['acque_time']==154837 or dic[key1]['actual_time_stamp']==actual_time_stamp) and ('800').startswith(dic[key1]['series_number'])]

In [ ]:
b_vals=[]
for i in range(len(big_dic_df)):
    dic=ast.literal_eval(big_dic_df['bval_big_dic'][i])
    b_vals.extend(list(dic.keys()))
    if('1000' in list(dic.keys())):
        print('bval 1000 for patient:{}'.format(big_dic_df['patient id'][i]))
    if('800' in list(dic.keys())):
        print('bval 800 for patient:{}'.format(big_dic_df['patient id'][i]))

In [ ]:
set(dwi_info_all.loc[dwi_info_all['manufacturer'].str.contains('SIEMENS',case=False),'patient id'].unique())-set(dwi_info_all.loc[dwi_info_all['series description'].str.contains('WIP',case=False),'patient id'].unique())

In [ ]:
set(dwi_info_all.loc[dwi_info_all['manufacturer'].str.contains('SIEMENS',case=False),'patient id'].unique())

In [ ]:
set(dwi_info_all.loc[dwi_info_all['series description'].str.contains('opt',case=False),'patient id'].unique())

In [ ]:
dwi_info_all.loc[dwi_info_all['patient id']==23742679]

# DCE

In [ ]:
DCE_not_desired=DWI_not_desired+['nw_sf8']

In [ ]:
info_csv.loc[(info_csv['n_files']>200)&(~info_csv['series description'].str.contains('|'.join(DCE_not_desired),case=False)),'series description'].unique()

In [ ]:
len(info_csv.loc[(info_csv['n_files']>200)&(~info_csv['series description'].str.contains('|'.join(DWI_not_desired),case=False))])

In [ ]:
patientw_w_DCE = set(info_csv.loc[(info_csv['n_files']>300),'patient id'])

In [ ]:
set(patients)-set(patientw_w_DCE)

# temp

In [ ]:
temp_df= pd.read_csv(os.path.join(S.root,"sheets","temp_stats.csv"))

In [ ]:
temp_df['types str']=temp_df['types'].apply(lambda x: str(x))
temp_df.groupby('types str')['patient id'].count()


# warning patients

In [ ]:
'23703200',
'23678469',
'23636822',
'23133176',
'23602410',
'23344041',
'23303423',
'18873901',
'14429740',
'12189049',
'08029696',
'28163350',
'28666188'

In [10]:
warning_patients_dwi=['23703200','23303423','23678469','18873901','23636822','23602410','08029696','23344047','23133176','14429740','12189049']

In [11]:
dwi_info_df.columns

Index(['acqu time', 'actual time stamp', 'b values', 'coil name',
       'manufacturer', 'modality', 'model name', 'n_files', 'patient id',
       'pixel spacing', 'protocol', 'series description', 'series number',
       'series uid', 'slice tickness', 'station', 'study description',
       'study uid'],
      dtype='object')

In [12]:
dwi_info_df.loc[dwi_info_df['patient id'].isin(warning_patients_dwi)].groupby('manufacturer')['patient id'].nunique()

manufacturer
GE MEDICAL SYSTEMS    11
Name: patient id, dtype: int64

In [13]:
dwi_info_df.loc[dwi_info_df['patient id'].isin(warning_patients_dwi)].groupby('model name')['patient id'].nunique()

model name
SIGNA HDx    11
Name: patient id, dtype: int64

In [14]:
dwi_info_df.loc[dwi_info_df['patient id'].isin(warning_patients_dwi)].groupby('station')['patient id'].nunique()

station
B221GE30    7
MR23T       4
Name: patient id, dtype: int64

In [17]:
dwi_info_df.loc[dwi_info_df['patient id'].isin(warning_patients_dwi)].groupby('protocol')['patient id'].nunique()

protocol
DCE_DWI T1 MAP MULTI FL/    2
PRO DCE DWI 08-21-08/7      1
PRO DCE DWI/7               8
Name: patient id, dtype: int64

In [20]:
dwi_info_df.loc[(dwi_info_df['protocol']=='PRO DCE DWI 08-21-08/7') &(~dwi_info_df['patient id'].isin(warning_patients_dwi))]

,acqu time,actual time stamp,b values,coil name,manufacturer,modality,model name,n_files,patient id,pixel spacing,protocol,series description,series number,series uid,slice tickness,station,study description,study uid
255,90704.0,"[184, 129, 174, 72]","[b'500\\8\\0\\0 ', b'0\\8\\0\\0 ']",EIS_eCoil Torso,GE MEDICAL SYSTEMS,MR,SIGNA HDx,24,20811956,"[1.0938, 1.0938]",PRO DCE DWI 08-21-08/7,Ax DWI b500 5mm 28cm no assett#2,7,1.2.840.113619.2.207.3596.11861984.25740.1219404288.433,5.0,MR23T,PELVIS W&W/O,1.2.124.113532.170.223.162.178.20080819.65852.4482573
256,90704.0,"[184, 129, 174, 72]",[b'500\\8\\0\\0 '],EIS_eCoil Torso,GE MEDICAL SYSTEMS,MR,SIGNA HDx,12,20811956,"[1.0938, 1.0938]",PRO DCE DWI 08-21-08/7,Exponential Apparent Diffusion Coefficient,700,1.2.840.113619.2.5.20201724.700.1219413018.36,5.0,MR23T,PELVIS W&W/O,1.2.124.113532.170.223.162.178.20080819.65852.4482573
257,90704.0,"[184, 129, 174, 72]",[b'500\\8\\0\\0 '],EIS_eCoil Torso,GE MEDICAL SYSTEMS,MR,SIGNA HDx,12,20811956,"[1.0938, 1.0938]",PRO DCE DWI 08-21-08/7,Apparent Diffusion Coefficient (mm?/s),701,1.2.840.113619.2.5.20201724.701.1219413018.36,5.0,MR23T,PELVIS W&W/O,1.2.124.113532.170.223.162.178.20080819.65852.4482573
325,110214.0,"[54, 76, 202, 72]","[b'500\\8\\0\\0 ', b'0\\8\\0\\0 ']",EIS_eCoil Torso,GE MEDICAL SYSTEMS,MR,SIGNA HDx,24,22936470,"[0.7031, 0.7031]",PRO DCE DWI 08-21-08/7,Ax DWI b500 5mm 28cm no assett#2,7,1.2.840.113619.2.207.3596.11861984.28291.1221219271.120,5.0,MR23T,PELVIS W&W/O,1.2.124.113532.170.223.162.178.20080908.163740.4606264
326,110214.0,"[54, 76, 202, 72]",[b'500\\8\\0\\0 '],EIS_eCoil Torso,GE MEDICAL SYSTEMS,MR,SIGNA HDx,12,22936470,"[0.7031, 0.7031]",PRO DCE DWI 08-21-08/7,Exponential Apparent Diffusion Coefficient,700,1.2.840.113619.2.5.20212322.700.1221232953.184,5.0,MR23T,PELVIS W&W/O,1.2.124.113532.170.223.162.178.20080908.163740.4606264
327,110214.0,"[54, 76, 202, 72]",[b'500\\8\\0\\0 '],EIS_eCoil Torso,GE MEDICAL SYSTEMS,MR,SIGNA HDx,12,22936470,"[0.7031, 0.7031]",PRO DCE DWI 08-21-08/7,Apparent Diffusion Coefficient (mm?/s),701,1.2.840.113619.2.5.20212322.701.1221232953.184,5.0,MR23T,PELVIS W&W/O,1.2.124.113532.170.223.162.178.20080908.163740.4606264
343,145538.0,"[106, 143, 227, 72]","[b'0\\8\\0\\0 ', b'500\\8\\0\\0 ']",EIS_eCoil Torso,GE MEDICAL SYSTEMS,MR,SIGNA HDx,40,23263130,"[0.5469, 0.5469]",PRO DCE DWI 08-21-08/7,Ax DWI b500 5mm 28cm no assett#2,7,1.2.840.113619.2.207.3596.11861984.26783.1222857779.633,3.0,MR23T,PELVIS W&W/O,1.2.124.113532.170.223.162.178.20080926.92304.4731665
344,145538.0,"[106, 143, 227, 72]",[b'500\\8\\0\\0 '],EIS_eCoil Torso,GE MEDICAL SYSTEMS,MR,SIGNA HDx,20,23263130,"[0.5469, 0.5469]",PRO DCE DWI 08-21-08/7,Exponential Apparent Diffusion Coefficient,700,1.2.840.113619.2.5.20231616.700.1222889304.615,3.0,MR23T,PELVIS W&W/O,1.2.124.113532.170.223.162.178.20080926.92304.4731665
345,145538.0,"[106, 143, 227, 72]",[b'500\\8\\0\\0 '],EIS_eCoil Torso,GE MEDICAL SYSTEMS,MR,SIGNA HDx,20,23263130,"[0.5469, 0.5469]",PRO DCE DWI 08-21-08/7,Apparent Diffusion Coefficient (mm?/s),701,1.2.840.113619.2.5.20231616.701.1222889304.615,3.0,MR23T,PELVIS W&W/O,1.2.124.113532.170.223.162.178.20080926.92304.4731665


In [18]:
dwi_info_df.loc[~dwi_info_df['patient id'].isin(warning_patients_dwi)].groupby('protocol')['patient id'].nunique()

protocol
ADC                         11
AX DWI Resolve 0-1000        1
AX DWI Resolve 0-500         1
DCE W T1 MAPPING_/10        24
DCE W T1 MAPPING_/8         16
DCE W T1 MAPPING_/9         40
DCE_DWI T1 MAP MULTI FL/     1
DWI Resolve WIP 0-500       12
DWI Resolve WIP0-1400       12
PRO DCE DWI 08-21-08/7       3
PRO DCE DWI/7               28
PRO DCE W_T1 MAPPING AN/    63
PRO DCE W_T1 MAPPING AND     3
PROSTATE DCE  3 27 2013/    12
PROSTATE DCE  8 20 2012/    17
PROSTATE W BLOCH-SIEGER/     3
Prostate With e-coil C-/    11
Prostate w/e-coil C-/C+      1
Prostate w/o ecoil           1
Prostate with coil 11/14     1
Prostate with coil102313     1
Name: patient id, dtype: int64

In [ ]:
pu_opt_patients = dwi_info_all.loc[dwi_info['series description'].str.contains('pu|opt',case=False),'patient id'].unique()

In [ ]:
set([str(patient)for patient in pu_opt_patients])

In [ ]:
set(warning_adc_patients) - set([str(patient)for patient in pu_opt_patients])

In [ ]:
dwi_info_all.loc[dwi_info_all['patient id']==14429740]